In [1]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os

import prepare
import acquire
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier

In [2]:
telco_df = acquire.get_telco_data()
telco_df.head()


,Unnamed: 0,customer_id,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,...,contract_type_id.1,paperless_billing.1,gender.1,senior_citizen.1,partner.1,dependents.1,payment_type_id.1,monthly_charges.1,total_charges.1,signup_date
0,0,0002-ORFBO,2,1,2,Female,0,Yes,Yes,9,...,2,Yes,Female,0,Yes,Yes,2,65.6,593.3,2021-04-21 18:07:34
1,1,0003-MKNFE,2,1,1,Male,0,No,No,9,...,1,No,Male,0,No,No,2,59.9,542.4,2021-04-21 18:07:34
2,2,0004-TLHLJ,1,2,1,Male,0,No,No,4,...,1,Yes,Male,0,No,No,1,73.9,280.85,2021-09-21 18:07:34
3,3,0011-IGKFF,1,2,1,Male,1,Yes,No,13,...,1,Yes,Male,1,Yes,No,1,98.0,1237.85,2020-12-21 18:07:34
4,4,0013-EXCHZ,2,2,1,Female,1,Yes,No,3,...,1,Yes,Female,1,Yes,No,2,83.9,267.4,2021-10-21 18:07:34


In [3]:
telco_df.shape

(7043, 35)

In [4]:
telco_df.columns

Index(['Unnamed: 0', 'customer_id', 'payment_type_id',
       'internet_service_type_id', 'contract_type_id', 'gender',
       'senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
       'contract_type', 'internet_service_type', 'payment_type',
       'contract_type_id.1', 'paperless_billing.1', 'gender.1',
       'senior_citizen.1', 'partner.1', 'dependents.1', 'payment_type_id.1',
       'monthly_charges.1', 'total_charges.1', 'signup_date'],
      dtype='object')

In [5]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7043 non-null   int64  
 1   customer_id               7043 non-null   object 
 2   payment_type_id           7043 non-null   int64  
 3   internet_service_type_id  7043 non-null   int64  
 4   contract_type_id          7043 non-null   int64  
 5   gender                    7043 non-null   object 
 6   senior_citizen            7043 non-null   int64  
 7   partner                   7043 non-null   object 
 8   dependents                7043 non-null   object 
 9   tenure                    7043 non-null   int64  
 10  phone_service             7043 non-null   object 
 11  multiple_lines            7043 non-null   object 
 12  online_security           7043 non-null   object 
 13  online_backup             7043 non-null   object 
 14  device_p

In [6]:
telco_df.describe()

,Unnamed: 0,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges,contract_type_id.1,senior_citizen.1,payment_type_id.1,monthly_charges.1
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692,1.690473,0.162147,2.315633,64.761692
std,2033.283305,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047,0.833755,0.368612,1.148907,30.090047
min,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000,1.000000,0.000000,1.000000,18.250000
25%,1760.500000,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000,1.000000,0.000000,1.000000,35.500000
50%,3521.000000,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000,1.000000,0.000000,2.000000,70.350000
75%,5281.500000,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000,2.000000,0.000000,3.000000,89.850000
max,7042.000000,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000,3.000000,1.000000,4.000000,118.750000


In [7]:
telco_clean_df = prepare.clean_telco(telco_df)


In [8]:
telco_clean_df.head()

,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,charge_rank
0,1,2,Female,0,Yes,Yes,9,Yes,No,No,...,Yes,No,Yes,65.6,590.4,No,One year,DSL,Mailed check,b
1,1,1,Male,0,No,No,9,Yes,Yes,No,...,No,Yes,No,59.9,539.1,No,Month-to-month,DSL,Mailed check,b
2,2,1,Male,0,No,No,4,Yes,No,No,...,No,No,Yes,73.9,295.6,Yes,Month-to-month,Fiber optic,Electronic check,a
3,2,1,Male,1,Yes,No,13,Yes,No,No,...,Yes,Yes,Yes,98.0,1274.0,Yes,Month-to-month,Fiber optic,Electronic check,b
4,2,1,Female,1,Yes,No,3,Yes,No,No,...,Yes,No,Yes,83.9,251.7,Yes,Month-to-month,Fiber optic,Mailed check,a


In [9]:
telco_clean_df['charge_rank'] = pd.qcut(telco_clean_df['total_charges'], 4, labels = ['a','b','c','d'])
telco_clean_df

charges_corrected = telco_clean_df[telco_clean_df.total_charges != 0]
charges_corrected['total_charges'].describe()

count    7032.000000
mean     2283.147248
std      2264.703327
min        18.800000
25%       397.800000
50%      1394.575000
75%      3791.250000
max      8550.000000
Name: total_charges, dtype: float64

In [13]:
readable_telco = pd.get_dummies(telco_clean_df[['charge_rank', 'internet_service_type', 'gender', 'senior_citizen', 'partner', 
                                                'contract_type', 'dependents', 'phone_service','multiple_lines',
                                               'online_security', 'online_backup','device_protection','streaming_tv', 
                                                'paperless_billing','churn', 'contract_type','internet_service_type',
                                               'payment_type']],drop_first =  True)
readable_telco = readable_telco.astype(int)
readable_telco


,senior_citizen,charge_rank_b,charge_rank_c,charge_rank_d,internet_service_type_Fiber optic,gender_Male,partner_Yes,contract_type_One year,contract_type_Two year,dependents_Yes,...,streaming_tv_No internet service,streaming_tv_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,1,0,0,0,0,1,1,0,1,...,0,1,1,0,1,0,0,0,0,1
1,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,1,0,1,0
3,1,1,0,0,1,1,1,0,0,0,...,0,1,1,1,0,0,1,0,1,0
4,1,0,0,0,1,0,1,0,0,0,...,0,1,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
7039,0,0,1,0,1,1,1,0,0,0,...,0,0,1,1,0,0,1,0,1,0
7040,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7041,0,0,0,1,0,1,1,0,1,1,...,0,0,0,0,0,1,0,0,0,1


In [14]:
telco_train, telco_val, telco_test = prepare.train_val_test(readable_telco, 'churn_Yes')


In [15]:
telco_train

,senior_citizen,charge_rank_b,charge_rank_c,charge_rank_d,internet_service_type_Fiber optic,gender_Male,partner_Yes,contract_type_One year,contract_type_Two year,dependents_Yes,...,streaming_tv_No internet service,streaming_tv_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
5932,0,1,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,1,0,1,0
1194,0,1,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4613,0,0,0,0,0,0,0,0,1,0,...,0,1,1,0,0,1,0,0,0,1
4991,0,0,0,1,1,0,1,0,1,1,...,0,1,0,0,0,1,1,1,0,0
6488,0,1,0,0,1,0,1,0,0,0,...,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
5319,1,0,1,0,0,1,0,0,1,0,...,1,0,0,0,0,1,0,1,0,0
1013,1,1,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
182,0,1,0,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [16]:
x_train = telco_train.drop(columns = ['churn_Yes'])
y_train = telco_train['churn_Yes']
x_val = telco_val.drop(columns = ['churn_Yes'])
y_val = telco_val['churn_Yes']
x_test = telco_test.drop(columns = ['churn_Yes'])
y_test = telco_test['churn_Yes']

# Random Forest

In [17]:
seed = 99

rf_telco = RandomForestClassifier(max_depth = 10, random_state = seed, min_samples_leaf = 1, max_samples = 0.5)
bsl = 1-(y_train.sum()/len(y_train))
bsl

0.7342543681430314

In [18]:
for1 = rf_telco.fit(x_train, y_train)


In [19]:
importance_df = pd.DataFrame({'feature': x_train.columns.tolist(), 
              'importance': rf_telco.feature_importances_})

In [20]:
importance_df.sort_values('importance', ascending = False)


,feature,importance
26,payment_type_Electronic check,0.086769
8,contract_type_Two year,0.079397
14,online_security_Yes,0.058981
4,internet_service_type_Fiber optic,0.054734
7,contract_type_One year,0.051771
23,contract_type_Two year,0.050711
3,charge_rank_d,0.049093
24,internet_service_type_Fiber optic,0.046152
21,paperless_billing_Yes,0.043231
22,contract_type_One year,0.041398


In [21]:
print(rf_telco.score(x_train, y_train),rf_telco.score(x_val, y_val))


0.8378707842340511 0.7772511848341233


In [22]:
train_preds = rf_telco.predict(x_train)


In [23]:
pd.DataFrame(classification_report(y_train, train_preds, output_dict = True))


,0,1,accuracy,macro avg,weighted avg
precision,0.862887,0.744722,0.837871,0.803804,0.831485
recall,0.926397,0.593272,0.837871,0.759835,0.837871
f1-score,0.893515,0.660426,0.837871,0.776970,0.831572
support,3614.000000,1308.000000,0.837871,4922.000000,4922.000000


In [24]:
metrics = []


def depth_n_leaf():
    msl = 1
    i = 3
    while msl <= 10:
        rf = RandomForestClassifier(max_depth = i,
                                    random_state = seed, min_samples_leaf = msl, max_samples = 0.5)
        rf.fit(x_train, y_train)
        train_acc = rf.score(x_train, y_train)
        val_acc = rf.score(x_val, y_val)
        output = {'max_depth': i, 'min_samples_leaf': msl, 'train_accuracy': train_acc, 
                  'validate_accuracy': val_acc}
        metrics.append(output)
        i = i +1
        if i == 11:
            msl = msl + 1
            i = 3


        
depth_n_leaf() 

metrics = pd.DataFrame(metrics)
metrics['difference'] = (metrics['train_accuracy'] - metrics['validate_accuracy'])    
metrics

,max_depth,min_samples_leaf,train_accuracy,validate_accuracy,difference
0,3,1,0.778545,0.761137,0.017408
1,4,1,0.782609,0.767773,0.014836
2,5,1,0.792564,0.784834,0.007730
3,6,1,0.795205,0.787678,0.007527
4,7,1,0.807192,0.789573,0.017619
5,8,1,0.812271,0.786730,0.025542
6,9,1,0.824462,0.784834,0.039627
7,10,1,0.837871,0.777251,0.060620
8,3,2,0.778545,0.761137,0.017408
9,4,2,0.782609,0.767773,0.014836


# Decision Tree


In [25]:
clf = DecisionTreeClassifier(max_depth = 3, random_state = seed)


In [26]:
clf.fit(x_train, y_train)


DecisionTreeClassifier(max_depth=3, random_state=99)

In [27]:
val_preds = clf.predict(x_val)


In [28]:
print(classification_report(y_val, val_preds))


              precision    recall  f1-score   support

           0       0.80      0.93      0.86       775
           1       0.66      0.36      0.46       280

    accuracy                           0.78      1055
   macro avg       0.73      0.65      0.66      1055
weighted avg       0.76      0.78      0.76      1055



In [29]:
clf.score(x_train, y_train)


0.7903291344981714

In [30]:
big_clf = DecisionTreeClassifier(max_depth = 6, random_state = 99)


In [31]:
big_clf.fit(x_train, y_train)


DecisionTreeClassifier(max_depth=6, random_state=99)

In [32]:
big_clf.score(x_train, y_train)


0.8061763510767981

In [33]:
metrics = []

for i in range (1, 25):
    tree = DecisionTreeClassifier(max_depth = i, random_state = 99)
    tree = tree.fit(x_train, y_train)
    in_sample_accuracy = tree.score(x_train, y_train)
    out_of_sample_accuracy = tree.score(x_val, y_val)
    output = {'max_dept': i, 'train_accuracy': in_sample_accuracy, 'validate_accuracy': out_of_sample_accuracy}
    
    metrics.append(output)
    
df= pd.DataFrame(metrics)
df['difference'] = df.train_accuracy - df.validate_accuracy
df

,max_dept,train_accuracy,validate_accuracy,difference
0,1,0.734254,0.734597,-0.000343
1,2,0.770215,0.773460,-0.003244
2,3,0.790329,0.780095,0.010234
3,4,0.790735,0.784834,0.005901
4,5,0.795002,0.789573,0.005429
5,6,0.806176,0.781991,0.024186
6,7,0.812271,0.785782,0.026489
7,8,0.824665,0.781991,0.042674
8,9,0.839293,0.775355,0.063938
9,10,0.854531,0.753555,0.100976


# KNN

In [34]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train, y_train)

KNeighborsClassifier()